# 02. Feature Engineering

This notebook creates user profiles based on listening history and audio features, preparing data for clustering analysis.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')

## Load Data

In [ ]:
# Load datasets
listening_history = pd.read_csv('../data/raw/listening_history.csv')
track_features = pd.read_csv('../data/raw/track_features.csv')

# Convert timestamp
listening_history['datetime'] = pd.to_datetime(listening_history['timestamp'])

print(f"Listening events: {listening_history.shape[0]:,}")
print(f"Unique users: {listening_history['user_id'].nunique():,}")
print(f"Unique tracks: {listening_history['track_id'].nunique():,}")

## Create User-Track Play Count Matrix

In [ ]:
# Calculate play counts per user-track pair
user_track_plays = listening_history.groupby(['user_id', 'track_id']).size().reset_index(name='play_count')

# Filter users with minimum activity
MIN_USER_PLAYS = 20
user_total_plays = user_track_plays.groupby('user_id')['play_count'].sum()
active_users = user_total_plays[user_total_plays >= MIN_USER_PLAYS].index

print(f"Active users (>={MIN_USER_PLAYS} plays): {len(active_users):,}")

# Filter data to active users
user_track_plays = user_track_plays[user_track_plays['user_id'].isin(active_users)]

## Audio Feature Aggregation

In [ ]:
# Define audio features to use
audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 
                  'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

# Check which features are available
available_features = [f for f in audio_features if f in track_features.columns]
print(f"Available audio features: {available_features}")

# Merge play counts with track features
user_track_features = user_track_plays.merge(
    track_features[['track_id'] + available_features], 
    on='track_id', 
    how='left'
)

# Check feature coverage
coverage = user_track_features[available_features].notna().all(axis=1).mean() * 100
print(f"\nFeature coverage: {coverage:.1f}%")

In [ ]:
# Calculate weighted average features per user
def calculate_weighted_features(df, features, weight_col='play_count'):
    """
    Calculate weighted average of features based on play counts
    """
    user_features = {}
    
    for user_id in df['user_id'].unique():
        user_data = df[df['user_id'] == user_id]
        
        # Remove tracks with missing features
        user_data = user_data.dropna(subset=features)
        
        if len(user_data) > 0:
            weights = user_data[weight_col]
            weighted_features = {}
            
            for feature in features:
                weighted_avg = np.average(user_data[feature], weights=weights)
                weighted_features[f'avg_{feature}'] = weighted_avg
                
                # Also calculate std deviation for diversity
                if len(user_data) > 1:
                    weighted_features[f'std_{feature}'] = np.sqrt(
                        np.average((user_data[feature] - weighted_avg)**2, weights=weights)
                    )
                else:
                    weighted_features[f'std_{feature}'] = 0
            
            user_features[user_id] = weighted_features
    
    return pd.DataFrame.from_dict(user_features, orient='index')

# Calculate user audio profiles
user_audio_profiles = calculate_weighted_features(user_track_features, available_features)
user_audio_profiles.index.name = 'user_id'
user_audio_profiles = user_audio_profiles.reset_index()

print(f"User audio profiles shape: {user_audio_profiles.shape}")
user_audio_profiles.head()

## Listening Behavior Features

In [ ]:
# Calculate listening behavior features
behavior_features = []

for user_id in active_users:
    user_history = listening_history[listening_history['user_id'] == user_id]
    user_plays = user_track_plays[user_track_plays['user_id'] == user_id]
    
    features = {
        'user_id': user_id,
        'total_plays': len(user_history),
        'unique_tracks': user_plays['track_id'].nunique(),
        'avg_plays_per_track': user_plays['play_count'].mean(),
        'play_count_std': user_plays['play_count'].std(),
        'track_diversity': user_plays['track_id'].nunique() / len(user_history),
    }
    
    # Temporal features
    user_history['hour'] = user_history['datetime'].dt.hour
    user_history['day_of_week'] = user_history['datetime'].dt.dayofweek
    
    # Most active hour
    hour_counts = user_history['hour'].value_counts()
    features['peak_hour'] = hour_counts.index[0] if len(hour_counts) > 0 else 0
    features['hour_diversity'] = len(hour_counts) / 24
    
    # Weekend vs weekday ratio
    weekend_plays = user_history[user_history['day_of_week'].isin([5, 6])].shape[0]
    features['weekend_ratio'] = weekend_plays / len(user_history)
    
    # Artist concentration
    if 'artist_name' in track_features.columns:
        user_artists = user_plays.merge(track_features[['track_id', 'artist_name']], on='track_id')
        artist_plays = user_artists.groupby('artist_name')['play_count'].sum()
        features['unique_artists'] = artist_plays.shape[0]
        features['artist_concentration'] = artist_plays.nlargest(5).sum() / artist_plays.sum()
    
    behavior_features.append(features)

user_behavior = pd.DataFrame(behavior_features)
print(f"User behavior features shape: {user_behavior.shape}")
user_behavior.head()

## Genre Preferences (if available)

In [ ]:
# Extract genre preferences if genre data is available
if 'genre' in track_features.columns:
    # Get user-genre play counts
    user_genre_data = user_track_plays.merge(
        track_features[['track_id', 'genre']], 
        on='track_id', 
        how='left'
    )
    
    # Calculate genre distributions per user
    user_genre_prefs = user_genre_data.groupby(['user_id', 'genre'])['play_count'].sum().unstack(fill_value=0)
    
    # Normalize to get genre preferences (percentages)
    user_genre_prefs = user_genre_prefs.div(user_genre_prefs.sum(axis=1), axis=0)
    
    # Keep top genres only to reduce dimensionality
    top_genres = user_genre_prefs.sum().nlargest(20).index
    user_genre_prefs = user_genre_prefs[top_genres]
    
    # Add prefix to column names
    user_genre_prefs.columns = [f'genre_{col}' for col in user_genre_prefs.columns]
    user_genre_prefs = user_genre_prefs.reset_index()
    
    print(f"User genre preferences shape: {user_genre_prefs.shape}")
else:
    print("Genre information not available")
    user_genre_prefs = None

## Combine All Features

In [ ]:
# Merge all feature sets
user_features = user_behavior.merge(user_audio_profiles, on='user_id', how='inner')

if user_genre_prefs is not None:
    user_features = user_features.merge(user_genre_prefs, on='user_id', how='left')
    # Fill missing genre values with 0
    genre_cols = [col for col in user_features.columns if col.startswith('genre_')]
    user_features[genre_cols] = user_features[genre_cols].fillna(0)

print(f"Combined user features shape: {user_features.shape}")
print(f"\nFeature categories:")
print(f"- Behavior features: {len([col for col in user_features.columns if col in user_behavior.columns])}")
print(f"- Audio features: {len([col for col in user_features.columns if 'avg_' in col or 'std_' in col])}")
if user_genre_prefs is not None:
    print(f"- Genre features: {len([col for col in user_features.columns if col.startswith('genre_')])}")

## Feature Scaling and Transformation

In [ ]:
# Separate user_id from features
feature_columns = [col for col in user_features.columns if col != 'user_id']
X = user_features[feature_columns]

# Check for highly skewed features
skewness = X.apply(lambda x: stats.skew(x))
highly_skewed = skewness[abs(skewness) > 1].index.tolist()

print(f"Highly skewed features ({len(highly_skewed)}):")
for feat in highly_skewed[:5]:
    print(f"  {feat}: skewness = {skewness[feat]:.2f}")

# Apply log transformation to highly skewed features
X_transformed = X.copy()
for feat in highly_skewed:
    if X_transformed[feat].min() > 0:  # Only if all values are positive
        X_transformed[feat] = np.log1p(X_transformed[feat])

In [ ]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_transformed)
X_scaled_df = pd.DataFrame(X_scaled, columns=feature_columns)

# Visualize scaled feature distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

sample_features = ['total_plays', 'unique_tracks', 'avg_danceability', 
                   'avg_energy', 'track_diversity', 'artist_concentration']
sample_features = [f for f in sample_features if f in X_scaled_df.columns][:6]

for i, feat in enumerate(sample_features):
    axes[i].hist(X_scaled_df[feat], bins=50, edgecolor='black', alpha=0.7)
    axes[i].set_title(f'{feat} (scaled)')
    axes[i].set_xlabel('Scaled Value')
    axes[i].set_ylabel('Count')

plt.tight_layout()
plt.show()

## Dimensionality Reduction

In [ ]:
# Apply PCA to understand feature importance
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Plot explained variance
cumsum_var = np.cumsum(pca.explained_variance_ratio_)
n_components_90 = np.argmax(cumsum_var >= 0.9) + 1

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
         pca.explained_variance_ratio_, 'bo-', label='Individual')
plt.plot(range(1, len(cumsum_var) + 1), 
         cumsum_var, 'ro-', label='Cumulative')
plt.axhline(y=0.9, color='g', linestyle='--', label='90% threshold')
plt.axvline(x=n_components_90, color='g', linestyle='--')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('PCA Explained Variance')
plt.legend()
plt.grid(True)
plt.show()

print(f"Components needed for 90% variance: {n_components_90}")
print(f"Original features: {X_scaled.shape[1]}")

In [ ]:
# Analyze top features for first 3 principal components
n_top_features = 10

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i in range(3):
    # Get feature loadings for this component
    loadings = pd.Series(pca.components_[i], index=feature_columns)
    top_features = loadings.abs().nlargest(n_top_features)
    
    # Plot
    loadings[top_features.index].plot(kind='barh', ax=axes[i])
    axes[i].set_title(f'PC{i+1} Top Features (Var: {pca.explained_variance_ratio_[i]:.1%})')
    axes[i].set_xlabel('Loading')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Create Final Feature Sets

In [ ]:
# Create different feature sets for different purposes

# 1. Full feature set (scaled)
user_features_scaled = user_features.copy()
user_features_scaled[feature_columns] = X_scaled

# 2. PCA-reduced features
pca_reducer = PCA(n_components=n_components_90)
X_pca_reduced = pca_reducer.fit_transform(X_scaled)
pca_columns = [f'PC{i+1}' for i in range(n_components_90)]
user_features_pca = pd.DataFrame(X_pca_reduced, columns=pca_columns)
user_features_pca['user_id'] = user_features['user_id'].values

# 3. Selected important features only
# Select based on PCA loadings and domain knowledge
important_features = ['user_id', 'total_plays', 'unique_tracks', 'track_diversity',
                     'avg_danceability', 'avg_energy', 'avg_valence', 
                     'std_danceability', 'std_energy', 'std_valence']

# Add artist concentration if available
if 'artist_concentration' in user_features.columns:
    important_features.append('artist_concentration')

# Add top genre features if available
if user_genre_prefs is not None:
    genre_cols = [col for col in user_features.columns if col.startswith('genre_')]
    genre_importance = user_features[genre_cols].mean().nlargest(5)
    important_features.extend(genre_importance.index.tolist())

# Filter to available features
important_features = [f for f in important_features if f in user_features.columns]
user_features_selected = user_features[important_features].copy()

# Scale selected features
selected_feature_cols = [col for col in important_features if col != 'user_id']
scaler_selected = StandardScaler()
user_features_selected[selected_feature_cols] = scaler_selected.fit_transform(
    user_features_selected[selected_feature_cols]
)

print("Feature sets created:")
print(f"1. Full features (scaled): {user_features_scaled.shape}")
print(f"2. PCA features: {user_features_pca.shape}")
print(f"3. Selected features: {user_features_selected.shape}")

## Feature Correlation Analysis

In [ ]:
# Analyze correlations in selected features
if len(selected_feature_cols) <= 15:
    plt.figure(figsize=(12, 10))
    corr_matrix = user_features_selected[selected_feature_cols].corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=1, cbar_kws={"shrink": 0.8},
                fmt='.2f')
    plt.title('Feature Correlations (Selected Features)')
    plt.tight_layout()
    plt.show()
else:
    print("Too many features for correlation heatmap")

## Save Engineered Features

In [ ]:
# Save all feature sets
user_features.to_csv('../data/processed/user_features_raw.csv', index=False)
user_features_scaled.to_csv('../data/processed/user_features_scaled.csv', index=False)
user_features_pca.to_csv('../data/processed/user_features_pca.csv', index=False)
user_features_selected.to_csv('../data/processed/user_features_selected.csv', index=False)

# Save scalers for later use
import joblib
joblib.dump(scaler, '../models/feature_scaler_full.pkl')
joblib.dump(scaler_selected, '../models/feature_scaler_selected.pkl')
joblib.dump(pca_reducer, '../models/pca_reducer.pkl')

print("Feature engineering complete!")
print("\nSaved files:")
print("- user_features_raw.csv: Original features")
print("- user_features_scaled.csv: All features, standardized")
print("- user_features_pca.csv: PCA-reduced features")
print("- user_features_selected.csv: Selected important features")
print("\nSaved models:")
print("- feature_scaler_full.pkl")
print("- feature_scaler_selected.pkl")
print("- pca_reducer.pkl")

## Feature Summary

In [ ]:
# Create feature summary
print("=== FEATURE ENGINEERING SUMMARY ===")
print(f"\nTotal users processed: {len(user_features):,}")
print(f"Total features created: {len(feature_columns)}")
print(f"\nFeature categories:")
print(f"- Listening behavior: {len([f for f in feature_columns if f in user_behavior.columns])}")
print(f"- Audio preferences: {len([f for f in feature_columns if 'avg_' in f or 'std_' in f])}")
if user_genre_prefs is not None:
    print(f"- Genre preferences: {len([f for f in feature_columns if f.startswith('genre_')])}")
print(f"\nDimensionality reduction:")
print(f"- Original dimensions: {len(feature_columns)}")
print(f"- PCA components (90% var): {n_components_90}")
print(f"- Selected features: {len(selected_feature_cols)}")

# Feature statistics
print("\n=== KEY FEATURE STATISTICS ===")
key_stats = user_features[['total_plays', 'unique_tracks', 'track_diversity']].describe()
print(key_stats.round(2))